In [11]:
import sys
import os
import numpy as np
import pandas as pd

# need to be able to import src
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# absolute path to the src directory
src_path = os.path.abspath("../src")

# Add src directory to system path
sys.path.append(src_path)

# get the /MLCBAss1/Assignment-1 path
ass_path=sys.path[-2]

print(ass_path)


%load_ext autoreload
%autoreload 2

from functions import *

/home/sdi1700187/MLCBAss1/Assignment-1
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# we load the two datasets
dev_set_df=pd.read_csv("../data/development_final_data.csv")
val_set_df=pd.read_csv("../data/evaluation_final_data.csv")

dev_set_df=dev_set_df.drop(labels=['Unnamed: 0'],axis=1)
val_set_df=val_set_df.drop(labels=['Unnamed: 0'],axis=1)

dev_set_df.head()
# val_set_df.head()
bacteria=get_bacteria(dev_set_df,mode='analysis')
bacteria_names=bacteria.columns
y_train=dev_set_df["BMI"]
x_train=scale_data(bacteria,bacteria_names)


In [13]:
models_dir="../models"

model_io=IO(models_dir)

baseline_predictor=DefaultPredictor()

In [14]:
# Baseline models

# train models
baseline_models=baseline_predictor.train_models(x_train=x_train,y_train=y_train)

# save models
for name,model in baseline_models.items():
    model_io.save(model,name,suf='bsln')